# Airport Data Converter
##### Code by Alessandro Romancino

This notebook was made just to show **how I imported all the data from the airports file** given by prof. Micciché and how I made the files which are present in the [source/data](../source/data/) folder.

I think this is useful because there are few good resources online on how to efficiently import big files like the one given (~238 MB) and how to manipulate the data.

The data was split into three files:
  - a [JSON file](https://www.json.org/json-en.html) containing the data relative to the coordinates of the airports (basically a [Python dictionary](https://docs.python.org/3/library/stdtypes.html#typesmapping))
  - An [LGL formatted file](https://github.com/TheOpteProject/LGL/) containing the edge list with the weights and another one with the unweighted network

In order to reproduce this notebook you just need to place the `20170901_m1.so6` file in the same folder as this notebook, then you can play with it. _The file was too large for GitHub so I uploaded it on my Google Drive with this link:_

https://drive.google.com/file/d/1WZNhskGxDEcETyJq3BZui5dGCPhhvyes/view?usp=share_link

## First Imports

For this notebook you will only need igraph, numpy and the preinstalled [json module](https://docs.python.org/3/library/json.html).

In [3]:
import numpy as np
import igraph as ig
import json

For our purposes we can import all the data in a numpy **ndarray** using the [`numpy.loadtxt`](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html) method.

We only need the columns indicating:
  1. Name of departing airport in the [ICAO format](https://en.wikipedia.org/wiki/ICAO_airport_code) **(1)**
  2. Name of arriving airport **(2)**
  3. Latitude of the departing point in the segment **(12)**
  4. Longitude of the departing point in the  segment **(13)**
  5. Latitude of the arriving point in the  segment **(14)**
  6. Longitude of the arriving point in the  segment **(15)**
  7. Segment number **(17)**

In [3]:
data = np.loadtxt(
    "20170901_m1.so6", delimiter=" ", usecols=(1, 2, 12, 13, 14, 15, 17), dtype=str
)
data

array([['ZZZZ', 'ZZZZ', '3382.000000', ..., '3393.000000', '194.000000',
        '1'],
       ['ZZZZ', 'ENUG', '4279.000000', ..., '4278.666667', '1335.050000',
        '1'],
       ['EPPW', 'EPPW', '3142.750000', ..., '3142.750000', '1071.133333',
        '1'],
       ...,
       ['LICJ', 'LIRF', '2524.650000', ..., '2519.266667', '727.300000',
        '36'],
       ['LICJ', 'LIRF', '2519.266667', ..., '2515.350000', '729.750000',
        '37'],
       ['LICJ', 'LIRF', '2515.350000', ..., '2508.016667', '734.333333',
        '38']], dtype='<U12')

## Segment Filtering

Every route is divided into multiple segments **numbered starting from 1**. We actually care about the first and last one containing the coordinates of the starting and ending airports, so we need to filter out these segments. We get the segment number array with the last column which we cast to integers.

We do so by first getting the segments array, and the difference array, calculated as `diff[i] = segments[i+1] - segments[i]`. In order to include also the last element we use the [`numpy.roll`](https://numpy.org/doc/stable/reference/generated/numpy.roll.html) function with _shift -1_, which **shifts all element** 1 ahead with periodic boundary conditions (last element is placed first).

In [4]:
segments = data[:, -1].astype("int32")
diff = np.roll(segments, -1) - segments
segments

array([ 1,  1,  1, ..., 36, 37, 38])

We first make masks by checking where `diff <= 0` to get the end points (the last segment is the highest number then we start with a new segment with number 1). The starting segments are found where `segments == 1`.

We use [`numpy.flatnonzero`](https://numpy.org/doc/stable/reference/generated/numpy.flatnonzero.html) to find the indices where the array is `True`.

At the end we check if both arrays are equal in size.

In [5]:
end = np.flatnonzero(diff <= 0)
start = np.flatnonzero(segments == 1)
print(len(end), len(start))

36798 36798


Using these indices we now get the names and coordinates of the start and end points. We also convert the coordinates from `str` to `float` and divide by 60 because the coordinates were given in **minutes**.

In [6]:
start_name = data[start][:, 0]
start_coords = data[start][:, [2, 3]].astype("float32") / 60

end_name = data[end][:, 1]
end_coords = data[end][:, [4, 5]].astype("float32") / 60

## JSON coordinate dictionary

Now to create our coordinates database we will make a dictionary from the name and coordinates. We create both the dictionaries using a simple dictionary comprehension (which is the same as a list comprehension).

We can now merge the two dictionaries to get a single dictionary that associate the airport ICAO name to the coordinates (elements of the first array are overridden but it doesn't matter).

In [7]:
start_dic = {name: f"[{lat}, {long}]" for name, (lat, long) in zip(start_name, start_coords)}
end_dic = {name: f"[{lat}, {long}]" for name, (lat, long) in zip(end_name, end_coords)}

loc_dic = start_dic | end_dic

In order to use the json dump method we already have converted the data to strings by using the so called **string representation** of the list of the coordinates.

We can now use the json module to dump everything to a file.

In [8]:
with open("airports_coords.json", "w") as fp:
    json.dump(loc_dic, fp)

If we want to import the data one needs to load the file with the json module and then we can use the `json.loads` function to **convert the string representation back to the original list**.

In [9]:
with open("airports_coords.json", "r") as fp:   
    my_dump = json.load(fp)
    airport = {key: json.loads(val) for key, val in my_dump.items()}
airport

{'ZZZZ': [71.31666564941406, 22.233333587646484],
 'EPPW': [52.37916564941406, 17.852222442626953],
 'EDDF': [50.03333282470703, 8.570555686950684],
 'KEWR': [40.6966667175293, -74.1683349609375],
 'EDDM': [48.35388946533203, 11.786111831665039],
 'OTHH': [25.274444580078125, 51.608333587646484],
 'LEMD': [40.47222137451172, -3.560833215713501],
 'LPPR': [41.23555374145508, -8.678055763244629],
 'LFPO': [48.723331451416016, 2.3794443607330322],
 'LMML': [35.85749816894531, 14.477499961853027],
 'FAOR': [-26.1391658782959, 28.246112823486328],
 'OMDB': [25.24916648864746, 55.36000061035156],
 'EGBB': [52.45389175415039, -1.7480555772781372],
 'LPPT': [38.774166107177734, -9.134166717529297],
 'EPWA': [52.16583251953125, 20.967222213745117],
 'OMAA': [24.433332443237305, 54.650001525878906],
 'LEST': [42.89638900756836, -8.415277481079102],
 'EHAM': [52.30805587768555, 4.764166831970215],
 'KTPA': [27.96666717529297, -82.53333282470703],
 'EGPK': [55.50944519042969, -4.594444274902344],


## The edge list

We now want to get the edge list. We will then use various functions from the [igraph module](https://igraph.readthedocs.io/en/stable/) to import and export the data into `igraph.Graph` objects.

As we already said we only need one of the multiple segment present in the datasheet so we can just use the **start** mask to choose our segments.

We can now use the [`Graph.TupleList`](https://igraph.readthedocs.io/en/stable/api/igraph.Graph.html#TupleList) method which will create a [`igraph.Graph`](https://igraph.readthedocs.io/en/stable/api/igraph.Graph.html) object for us.

By default it will create an undirected multilink graph. The name of the nodes is added as a `igraph.Vertex` attribute with the name **name**. In order to access them one can use the [`igraph.VertexSeq`](https://igraph.readthedocs.io/en/stable/api/igraph.VertexSeq.html) object.

In [12]:
named_edges = data[start][:, [0, 1]]
airport_graph = ig.Graph.TupleList(named_edges, directed=False)
airport_graph.vs["name"]

['ZZZZ',
 'ENUG',
 'EPPW',
 'EDDF',
 'EGLL',
 'KEWR',
 'LFPG',
 'EDDM',
 'LRSB',
 'OTHH',
 'LDZA',
 'LEMD',
 'LEIB',
 'LPPR',
 'LSZH',
 'LFPO',
 'LPFR',
 'LMML',
 'LIML',
 'FAOR',
 'OMDB',
 'EGKK',
 'EGBB',
 'GCTS',
 'LIBD',
 'LPPT',
 'LIRF',
 'EPWA',
 'EPGD',
 'OMAA',
 'EGCC',
 'LEST',
 'GMMN',
 'EHAM',
 'MPTO',
 'KTPA',
 'EGPK',
 'EGGP',
 'GCXO',
 'GCLA',
 'KBOS',
 'KLAX',
 'EGNX',
 'EGAA',
 'WSSS',
 'EBBR',
 'ELLX',
 'LICD',
 'LICC',
 'ESPA',
 'LEPA',
 'LEMG',
 'KDAL',
 'EGLF',
 'LFRS',
 'LOWW',
 'EDDS',
 'UUEE',
 'EGNS',
 'ENTC',
 'ENBR',
 'EGKB',
 'LTFJ',
 'LATI',
 'HEGN',
 'LKPR',
 'LEBL',
 'LYTV',
 'ENVA',
 'ENGM',
 'UMKK',
 'UUDD',
 'LFBO',
 'LFRI',
 'EGPH',
 'BIKF',
 'ENKB',
 'EDLW',
 'LRCL',
 'LSGG',
 'LFSB',
 'LLBG',
 'EDFC',
 'EDQG',
 'EDDK',
 'EDDC',
 'OLBA',
 'GCRR',
 'UMMS',
 'OIIE',
 'LZIB',
 'LZKZ',
 'EDSB',
 'LIRN',
 'EGGW',
 'EKRK',
 'EKYT',
 'EDDT',
 'LEAL',
 'EHRD',
 'EINN',
 'LFAC',
 'LFRC',
 'LEZL',
 'LEAM',
 'LTAT',
 'ESOH',
 'ESST',
 'GCLP',
 'EPKT',
 'GMMH',
 

So now we will create 2 files containing the weighted undirected network (called **weighted**) and another one with a simplified network without weights, multilink or self-loops (called **simple**). One could also create the directed versions by calling `directed = True` in all the functions.

In order to transform the multilink into weights and to remove self-loops one uses the [`Graph.simplify`](https://igraph.readthedocs.io/en/stable/api/igraph.GraphBase.html#simplify) function.

We first create a copy of our graph (called **temp_graph**) with a new `Edge` attribute called **weights** initialized to 1 (there are multilinks). Then we use simplify to remove multilinks (but not self-loops) and combine the edge attributes by summing them up (counts the number of links).

In [13]:
temp_graph = airport_graph.copy()
temp_graph.es["weights"] = 1
weighted = temp_graph.simplify(multiple=True, loops=False, combine_edges="sum")
simple = airport_graph.copy().simplify(multiple=True, loops=True)

For the export I liked the `.ncol` data format used by the [LGL (Large Graph Layout) project](https://github.com/TheOpteProject/LGL/). This is both simple and human readable.

The format keeps the name of the vertices and permits both weighted and directed networks by using the [`Graph.write_ncol`](https://igraph.readthedocs.io/en/stable/api/igraph.GraphBase.html#write_ncol) for export and [`Graph.Read_Ncol`](https://igraph.readthedocs.io/en/stable/api/igraph.GraphBase.html#Read_Ncol) for import. By default it will use our **name** attribute and we specify our **weights** attribute.

In [16]:
simple.write_ncol("simple_airport.ncol", weights=None)
weighted.write_ncol("weighted_airport.ncol", weights="weights")

So one can now easily read the data and plot the graph for example (we will discuss visualization later...)

In [5]:
g = ig.Graph.Read_Ncol("simple_airport.ncol", directed=False)
ig.plot(g, layout="graphopt", vertex_size=3, bbox=(500, 250))